# InSAR Time Series Analysis using MintPy and HyP3 products

**Author:** Alex Lewandowski

Based on the [LosAngeles_time_series](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Hazards/LosAngeles_time_series.ipynb) notebook by Eric Fielding, David Bekaert, Heresh Fattahi and Zhang Yunjun, which uses an example ARIA dataset.



## Mapping surface deformation with InSAR time series

This notebook demonstrates how to create an InSAR time series with MintPy using an ASF HyP3 InSAR data stack, which can be ordered on [ASF Data Search/Vertex](https://search.asf.alaska.edu/#/) and prepared using the [Prepare_HyP3_InSAR_Stack_for_MintPy](https://github.com/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb) notebook.

It also explores how to assess the quality of the stack inversion, temporal coherence, and velocity errors.

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# 0: Notebook setup

**Import necessary packages**

In [ ]:
import os
from pathlib import Path
import zipfile

from ipyfilechooser import FileChooser

from osgeo import gdal
import numpy as np
import mintpy.view
import mintpy.tsview
import mintpy.plot_network
import mintpy.plot_transection
import mintpy.plot_coherence_matrix

import asf_notebook as asfn

**Select the directory holding your MintPy-ready HyP3 data stack**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
fc = FileChooser('/home/jovyan/notebooks')
display(fc)

**Define a project name and create a MintPy directory in which to store files output during our analysis**

In [ ]:
# define the work directory
work_path = Path(fc.selected_path)
print(f"Work directory: {work_path}")

# define a project name
proj_name = input("Enter a project name: ")

# define the MintPy time-series directory
mint_path = work_path/'MintPy'
if not mint_path.is_dir():
    mint_path.mkdir()
print(f"MintPy directory: {mint_path}")


# 1: smallbaselineApp.py overview

This application provides a workflow which includes several steps to invert a stack of unwrapped interferograms and apply different corrections to obtain ground displacement timeseries.  
The workflow consists of two main blocks:

* correcting unwrapping errors and inverting for the raw phase time-series (blue ovals),
* correcting for noise from different sources to obtain the displacement time-series (green ovals).

Some steps are optional, which are switched off by default (marked by dashed boundaries). Configuration parameters for each step are initiated with default values in a customizable text file: [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg). In this notebook, we will walk through some of these steps, for a complete example see the [MintPy repository](https://github.com/insarlab/MintPy).

<p align="left">
  <img width="600" src="../Hazards/NotebookAddons/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>

## 1.1: Processing steps of smallbaselineApp.py

The MintPy **smallbaselineApp.py** application provides a workflow to invert a stack of unwrapped interferograms and apply different (often optional) corrections to obtain ground displacement timeseries. A detailed overview of the options can be retrieved by involking the help option:

In [ ]:
!smallbaselineApp.py --help

## 1.2: Configuring processing parameters

The processing parameters for the **smallbaselineApp.py** are controlled through a configuration file. If no file is provided the default [smallbaselineApp.cfg](https://github.com/insarlab/MintPy/blob/master/mintpy/defaults/smallbaselineApp.cfg) configuration is used. We will create a custom config file with modified configuration parameters for this time-series analysis.

In [ ]:
config = f'''# vim: set filetype=cfg:
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {work_path}/*/*unw_phase_clip.tif
mintpy.load.corFile          = {work_path}/*/*corr_clip.tif
##---------geometry datasets:
mintpy.load.demFile          = {work_path}/*/*dem_clip.tif
mintpy.load.incAngleFile     = {work_path}/*/*inc_map_clip.tif
mintpy.load.waterMaskFile    = {work_path}/*/*water_mask_clip.tif
'''

config_path = mint_path/f'{proj_name}.txt'

with open(config_path, 'w') as f:
    f.write(config)
    print(f"config file path: {config_path}\n")
    
with open(config_path, 'r') as f:
    for line in f.readlines():
        print(line)

# 2: Small Baseline Time Series Analysis

**You can run every step in smallbaselineApp.py with one call, using the command in the cell below**

**For the purposes of this tutorial, we will run each step separately**

We will run the steps:
- load_data
- modify_network
- reference_point
- quick_overview
- invert_network
- correct_troposphere
- correct_topography
- residual_RMS
- reference_date
- velocity
- google_earth

Skipped steps include:
- correct_unwrap_error
- correct_LOD
- correct_SET
- deramp
- hdfeos5

Skipped steps will also be skipped if running the entire smallbaselineApp in the cell below.

In [ ]:
# This runs every step

# !smallbaselineApp.py --work-dir {mint_path}  {config_path}

## 2.1: Load Data

**Run the `load_data` step**

- If you get a missing 'Height' attribute error, you are missing a DEM, which is an available option when ordering HyP3 InSAR products


In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep load_data

The output of the loading step is an "inputs" directory containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.  
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.). 

In [ ]:
inputs_path = mint_path/'inputs'
!ls $inputs_path

<div class="alert alert-info">
<b>info.py :</b> 
To get general infomation about a MintPy product, run info.py on the file.   
</div>

In [ ]:
!info.py $inputs_path/ifgramStack.h5

## 2.2: Modify the Network

**Run the `modify_network` step**

- If a water mask was provided, this step generates waterMask.h5
- Modify the network based on temporal/perpendicular baselines, date, num of connections etc.

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep modify_network

## 2.3: Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- ifgramStack_coherence_spatialAvg.txt: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- Network.pdf: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- CoherenceMatrix.pdf shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
mintpy.plot_network.main([f'{inputs_path}/ifgramStack.h5'])

## 2.4: Set the Reference Point

**Run the `reference_point` step**

The interferometric phase is relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore, we need to reference all interferograms to a common reference pixel.

The `reference_point` step selects a common reference pixel for the stack of interferograms. The default approach of MintPy is to choose a pixel with the highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel.   

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_point

**Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the ifgramStack.h5 file. To see the attributes of the file run info.py**

In [ ]:
!info.py $inputs_path/ifgramStack.h5 | egrep 'REF_'

## 2.5: Run a Quick Overview

**Run the `quick_overview` step**

- Assess possible groud deformation using the velocity from traditional interferogram stacking 
    - *reference: Zebker et al. (1997, JGR)*
- Assess distribution of phase unwrapping error from the number of interferogram triplets with non-zero integer ambiguity of closure phase 
    - *reference: T_int in Yunjun et al. (2019, CAGEO). Related to section 3.2, equation (8-9) and Fig. 3d-e.*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep quick_overview

## 2.6: Inverting of the Small Baseline network

**Run the `invert_network` step**

- Invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date
- By default mintpy selects the first acquisition
- The estimated time-series is converted to distance change from radar to target and is provided in meters. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep invert_network

## 2.7: Correct for Tropospheric Propagation Delays

**Run the `correct_troposphere` step**

- Uses ECMWF [ERA5 climate reanalysis pressure data](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20)%20AND%20(%20%22Provider:%20Copernicus%20C3S%22%20))&text=pressure)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_troposphere 

## 2.8: Correct for DEM Errors

**Run the `correct_topography` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_topography

## 2.9: Calculate the Root Mean Square (RMS) of Residual Phase Time-Series for Each Acquisition

**Run the `residual_RMS` step**

- *reference: Yunjun et al. (2019, section 4.9 and 5.4)*
- To remove the long wavelength component in space, a phase ramp is removed for each acquisition
- Sets optimal reference date to date with min RMS
- Sets exclude dates (outliers) to dates with RMS > cutoff * median RMS (Median Absolute Deviation)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep residual_RMS

## 2.10: Reference the Entire Time-Series to One Date in Time

**Run the `reference_date` step**

- *reference: Yunjun et al. (2019, section 4.9)*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_date

## 2.11: Estimate The Long-Term Velocity Rate

**Run the `velocity` step**

The timeseries file contains three datasets:
- the `time-series` dataset, which is the interferometric range change for each acquisition relative to the reference acquisition
- the `date` dataset, which contains the acquisition date for each acquisition
- the `bperp` dataset, which contains the timeseries of the perpendicular baseline 

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep velocity

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 velocity -v -1 1'
mintpy.view.main(scp_args.split())

## 2.12: Create a kmz File

**Run the `google_earth` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep google_earth

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
    The line of sight (LOS) for this descending Sentinel-1 track is up and east from ground to radar.
</div>

# 3: Error analysis (signal vs noise)

Uncertainty of the ground displacement products derived from InSAR time-series, depends on the quality of the inversion of the stack of interferograms and the accuracy in separating the ground displacement from other components of the InSAR data. Therefore the definition of signal vs noise is different at the two main steps in MintPy:  

1) During the inversion: 
    At this step all systematic components of the interferometric phase (e.g., ground displacement, propagation delay, geometrical residuals caused by DEM or platform's orbit inaccuracy) are considered signal, while the interferometric phase decorrelation, phase unwrapping error and phase inconsistency are considered noise. 
    
2) After inversion: the ground displacement component of the time-serieses is signal, and everything else (including the propagation delay and geometrical residuals) are considered noise

Therefore we first discuss the possible sources of error during the inversion and the existing ways in MintPy to evaluate the quality of inversion and to improve the uncertainty of the inversion. Afterwards we explain the different components of the time-series and the different processing steps in MintPy to separate them from ground displacement signal.  


## 3.1: Quality of the inversion

The main sources of noise during the time-series inversion includes decorrelation, phase unwrapping error and the inconsistency of triplets of interferofgrams. Here we mainly focus on the decorrelation and unwrapping errors. We first show the existing quantities in MintPy to evaluate decorrelation and unwrapping errors and then discuss the existing ways in MintPy to reduce the decorrelation and unwrapping errors on the time-series inversion.

### 3.1.1: Average spatial coherence

Mintpy computes temporal average of spatial coherence of the entire stack as a potential ancillary measure to choose reliable pixels after time-series inversion.

In [ ]:
%matplotlib notebook
mintpy.view.main([f'{mint_path}/avgSpatialCoh.h5'])

### 3.1.2: Temporal coherence

In addition to timeseries.h5 which contains the time-series dataset, invert_network produces other quantities, which contain metrics to evaluate the quality of the inversion including temporalCoherence.h5. Temporal coherence represents the consistency of the timeseries with the network of interferograms. 

Temporal coherence varies from 0 to 1. Pixels with values closer to 1 are considered reliable and pixels with values closer to zero are considered unreliable. For a dense network of interferograms, a threshold of 0.7 may be used (Yunjun et al, 2019).

In [ ]:
%matplotlib notebook
mintpy.view.main([f'{mint_path}/temporalCoherence.h5'])

With both the spatial coherence and temporal coherence, we can see that the InSAR in the ports of Long Beach and Los Angeles have unstable phase, and the InSAR measurements there will be low quality.

## 3.2: Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

In [ ]:
%matplotlib notebook
scp_args = f'{mint_path}/velocity.h5 velocityStd -v 0 0.2'
mintpy.view.main(scp_args.split())

Note that the plot above is the velocity error, not the velocity. The errors generally increase with distance from the reference point and also increase for points with elevations different from the reference point because of topographically correlated water vapor variations that are especially strong in this area.

# 4: Plotting a Motion Transect 

**Select the area for which to plot a motion transect**

In [ ]:
%matplotlib notebook
image_file = f"{work_path}/raster_stack.vrt"
amp = list(work_path.glob('*/*_amp_clip.tif'))[0]
!gdalbuildvrt -separate $image_file -overwrite $amp
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

fig_xsize = 7.5
fig_ysize = 7.5
aoi = asfn.AOI_Selector(rasterstack, fig_xsize, fig_ysize)

In [ ]:
geotrans = img.GetGeoTransform()

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

In [ ]:
try:
    ul = geolocation(aoi.x1, aoi.y1, geotrans)
    lr = geolocation(aoi.x2, aoi.y2, geotrans)
#     print(f"aoi_coords in EPSG {utm}")
    print(f"upper left corner: {ul}")
    print(f"lower right corner: {lr}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))

In [ ]:
%matplotlib inline
scp_args = f'{mint_path}/velocity.h5 --start-lalo {ul[1]} {ul[0]} --end-lalo {lr[1]} {lr[0]}'
mintpy.plot_transection.main(scp_args.split())

# 5: Plot the Cumulative Displacement Map and Point Displacement Time Series

In [ ]:
%matplotlib notebook
mintpy.tsview.main([f'{mint_path}/timeseries.h5'])

 On this transect, the Portuguese Bend Landslide has an average velocity over the last five years that reaches more than 2.5 cm/year in the descending track radar LOS direction (note that the "zero" is about -5 so we have to subtract that first). By analyzing the velocity on the ascending track and combining the two LOS directions, then we could find out that the displacments are largely horizontal, westward (and southward that we can't see with InSAR).

# Reference material

- Mintpy reference: *Yunjun, Z., H. Fattahi, F. Amelung (2019), Small baseline InSAR time series analysis: unwrapping error correction and noise reduction, preprint doi:[10.31223/osf.io/9sz6m](https://eartharxiv.org/9sz6m/).*

- University of Miami online time-series viewer: https://insarmaps.miami.edu/

- Mintpy Github repository: https://github.com/insarlab/MintPy

<font face="Calibri" size="2"> <i>MintPy_Time_Series_From_Prepared_Data_Stack.ipynb - Version 0.1 - September 2021
<br>
</i>
</font>